In [1]:
import pandas as pd

data = pd.read_csv("puchkov.k@phystech.edu.csv", header=None)
data.sample(5)

,0
46,0.0
12,1.0
25,1.0
3,1.0
38,0.0


In [2]:
data.describe()

,0
count,52.000000
mean,0.442308
std,0.501506
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [25]:
array = data.values.T.reshape(data.values.T.shape[1]).astype(int)
array

array([1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0])

In [26]:
from collections import Counter

def ngrams_and_prefix_counts(tokens, n_max):
    # словарь n-грамм и их частот
    ngrams_counts = {}
    # словарь n-граммных префиксов и их частот
    prefix_counts = {}
    
    n = len(tokens)
    for i in range(n_max):
        ngrams_counts[i + 1] = Counter([tuple(tokens[j : j + i + 1]) for j in range(n - i)])
        prefix_counts[i + 1] = Counter([tuple(tokens[j : j + i] + ['*']) for j in range(n - i)])

    return ngrams_counts, prefix_counts

#### 2-грамма и ее частотная вероятность

In [32]:
def bigram_probas(ngram_counts, prefix_counts):
    p2 = {}
    for w in ngram_counts[2]:
        pre_w = tuple([w[0]] + ['*'])
        p2[u'{1}|{0}'.format(*w)] = ngram_counts[2][w] / prefix_counts[2][pre_w]
    return p2

In [33]:
ngram_counts, prefix_counts = ngrams_and_prefix_counts(list(array), 3)

### Оценка $\mathbb{P}(X_{i+1}= 0|X_{i}=0)$ методом максимального правдоподобия

In [36]:
p2 = bigram_probas(ngram_counts, prefix_counts)
round(p2['0|0'], 2)

0.5

### Отвергается ли нулевая гипотеза $\mathbb{P}(X_{i+1}= 0|X_{i}=0)=0.9$?

Введем статистику $T = \sum^{n - 1}_{i = 1}I[X_{i+1} = 0, X_i = 0]$. Нулевое распределение — биномиальное распределение $Bin(n-1, p)$.

In [56]:
ngram_counts[2]

Counter({(1, 0): 15, (0, 0): 14, (0, 1): 14, (1, 1): 8})

Для нашей последовательности $T = 14$.

In [71]:
import scipy.stats as st
from statsmodels.stats.proportion import proportions_chisquare


T = 14
n = 52
p0 = 0.9
st.binom_test(T, n, p0, alternative='less'), proportions_chisquare(T, n, p0)[1]

(4.2142634985542545e-27, 6.3305005986367e-52)

Полученные $p-value$ меньше 0,05, следовательно, нулевая гипотеза отвергается.

### Какова вероятность встретить $X_i=2$ с применением сглаживания Лапласа

In [48]:
voc = list(set(array))
voc.append(2)
f = []
for i in voc:
    f.append(len(array[array == i]))

p_2 = (f[2] + 1)/(sum(f) + len(voc) )
round(p_2, 2)

0.02